<a href="https://colab.research.google.com/github/Tarun-619/BIS-lab/blob/main/BIS_lab5_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Cuckoo Search Algorithm(CSA)

In [4]:
import random
import math

weights = [4, 8, 5, 3, 6, 7]
values = [12, 20, 16, 10, 18, 22]
capacity = 20
n_items = len(weights)
n_nests = 12
max_iter = 40
pa = 0.25

def fitness(solution):
    total_wt = sum(w for w, s in zip(weights, solution) if s == 1)
    total_val = sum(v for v, s in zip(values, solution) if s == 1)
    return total_val if total_wt <= capacity else 0

def repair(solution):
    while sum(w for w, s in zip(weights, solution) if s == 1) > capacity:
        ones = [i for i, s in enumerate(solution) if s == 1]
        if not ones:
            break
        solution[random.choice(ones)] = 0
    return solution

def generate_nest():
    return [random.randint(0, 1) for _ in range(n_items)]

def levy_flight(Lambda=1.5):
    sigma_u = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
              (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = random.gauss(0, sigma_u)
    v = random.gauss(0, 1)
    return u / (abs(v) ** (1 / Lambda))

def get_cuckoo(nest, best_nest):
    new_nest = []
    for xi, bi in zip(nest, best_nest):
        step = levy_flight()
        val = xi + step * (xi - bi)
        s = 1 / (1 + math.exp(-val))
        new_val = 1 if s > 0.5 else 0
        new_nest.append(new_val)
    return repair(new_nest)

def cuckoo_search():
    nests = [generate_nest() for _ in range(n_nests)]
    fitness_vals = [fitness(repair(nest)) for nest in nests]
    best_idx = fitness_vals.index(max(fitness_vals))
    best_nest = nests[best_idx][:]
    best_fitness = fitness_vals[best_idx]
    for it in range(1, max_iter + 1):
        for i in range(n_nests):
            new_nest = get_cuckoo(nests[i], best_nest)
            new_fit = fitness(new_nest)
            if new_fit > fitness_vals[i]:
                nests[i], fitness_vals[i] = new_nest, new_fit
        num_abandon = int(pa * n_nests)
        worst_idx = sorted(range(len(fitness_vals)), key=lambda k: fitness_vals[k])[:num_abandon]
        for idx in worst_idx:
            nests[idx] = generate_nest()
            fitness_vals[idx] = fitness(repair(nests[idx]))
        cur_best_idx = fitness_vals.index(max(fitness_vals))
        if fitness_vals[cur_best_idx] > best_fitness:
            best_fitness = fitness_vals[cur_best_idx]
            best_nest = nests[cur_best_idx][:]
        if it % 10 == 0:
            print(f"Iter {it}: Best val = {best_fitness}")
    return best_nest, best_fitness

if __name__ == "__main__":
    best_solution, best_value = cuckoo_search()
    total_weight = sum(w for w, s in zip(weights, best_solution) if s == 1)
    print("\n__final sol__")
    print(f"best nest: {best_solution}")
    print(f"total val: {best_value}")
    print(f"total weight: {total_weight}/{capacity}")


Iter 10: Best val = 62
Iter 20: Best val = 62
Iter 30: Best val = 62
Iter 40: Best val = 62

__final sol__
best nest: [1, 0, 0, 1, 1, 1]
total val: 62
total weight: 20/20
